In [1]:
import os
import torch
from openvoice import se_extractor
from openvoice.api import BaseSpeakerTTS, ToneColorConverter

/home/anubhav/anaconda3/envs/Clone/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ckpt_base = 'checkpoints_v3/base_speakers/EN'
ckpt_converter = 'checkpoints_v3/converter'
device="cuda:0" if torch.cuda.is_available() else "cpu"
output_dir = 'outputs_v3'

base_speaker_tts = BaseSpeakerTTS(f'{ckpt_base}/config.json', device=device)
base_speaker_tts.load_ckpt(f'{ckpt_base}/checkpoint.pth')

tone_color_converter = ToneColorConverter(f'{ckpt_converter}/config.json', device=device)
tone_color_converter.load_ckpt(f'{ckpt_converter}/checkpoint.pth')

os.makedirs(output_dir, exist_ok=True)

/home/anubhav/anaconda3/envs/Clone/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Loaded checkpoint 'checkpoints_v3/base_speakers/EN/checkpoint.pth'
missing/unexpected keys: [] []
Loaded checkpoint 'checkpoints_v3/converter/checkpoint.pth'
missing/unexpected keys: [] []


we can chose between en_default_se.pth , en_style_se.pth, en-au.pth, en-br.pth, en-deault.pth, en-india.pth, en-newest.pth, en-uspth, 

In [3]:
source_se = torch.load(f'{ckpt_base}/en-us.pth').to(device) #Here we select the accent

In [ ]:
**Try with different styles and speed.** The style can be controlled by the `speaker` parameter in the `base_speaker_tts.tts` method. Available choices: friendly, cheerful, excited, sad, angry, terrified, shouting, whispering. Note that the tone color embedding need to be updated. The speed can be controlled by the `speed` parameter. Let's try whispering with speed 0.9.


In [6]:
# saving a new embedding, from the user, to be used in the synthesis
import os
import torch
from pathlib import Path

# After extracting the target speaker embeddings
reference_speaker = 'resources/demo_speaker2.mp3'  # This is the voice you want to clone
target_se, audio_name = se_extractor.get_se(reference_speaker, tone_color_converter, target_dir='processed', vad=True)

# Create a directory to save speaker embeddings if it doesn't exist
embeddings_dir = 'speaker_embeddings'
os.makedirs(embeddings_dir, exist_ok=True)


# Save the target speaker embeddings
embedding_filename = Path(reference_speaker).stem  # Get filename without extension
embedding_path = f'{embeddings_dir}/{embedding_filename}_se.pth'
torch.save(target_se, embedding_path)
print(f"Speaker embedding saved to {embedding_path}")

# Later, you can load these embeddings instead of extracting them again
target_se = torch.load(embedding_path).to(device)

OpenVoice version: v1
[(0.0, 8.21), (9.39, 12.946), (13.262, 29.49225)]
after vad: dur = 27.995986394557825
Speaker embedding saved to speaker_embeddings/demo_speaker2_se.pth


In [ ]:
# To load the saved embeddings
embedding_path = f'speaker_embeddings/ryanreynolds_se.pth'
target_se = torch.load(embedding_path).to(device)

In [11]:
save_path = f'{output_dir}/output_en_freddie_cheerful.wav' #name of the output file

# Run the base speaker tts
text = "Steve walks warily down the street. With the brim pulled way down low. Ain't no sound but the sound of his feet. Machine guns ready to go"
src_path = f'{output_dir}/tmp.wav'
base_speaker_tts.tts(text, src_path, speaker='default', language='English', speed=0.9) #speaker arg = emotion

# Run the tone color converter
encode_message = "@MyShell"
tone_color_converter.convert(
    audio_src_path=src_path, 
    src_se=source_se, 
    tgt_se=target_se, 
    output_path=save_path,
    message=encode_message)